In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path


In [ ]:
BASE_DIR = Path.cwd()          # ml/
DATA_DIR = BASE_DIR.parent / "data"

def load_csv(name):
    return pd.read_csv(DATA_DIR / f"{name}.csv")


Data

In [ ]:
orders=load_csv("order_level")
fact=load_csv("fact")
products=load_csv("dim_products")

In [ ]:
products.head()

In [ ]:
fact.describe()


In [ ]:
fact=fact.merge(products[['product_id','product_category_name']],on="product_id")

In [ ]:
fact.head()

In [ ]:
new=orders[['order_id','price','delivery_time']]
new.head()

In [ ]:

new=new.merge(fact[['order_id','product_category_name','seller_id','order_purchase_timestamp','delivery_cost']],on='order_id',how='inner')


In [ ]:

new.head()

In [ ]:
new

In [ ]:
new.isna().sum()

In [ ]:
#new['delivery_cost'].nunique()
new=new[~new['delivery_time'].isna()].copy()
new.head()


In [ ]:
new['delivery_time'].describe()

In [ ]:
#new['is_late']=np.where(new['delivery_time']>15,1,0)
#new['is_late']=new['delivery_time'].map(lambda x : 1 if x >15 else 0)
new['is_late']=(new['delivery_time']>15).astype(int)

In [ ]:
new.head()

In [ ]:
# check the propostions of late orders
new['is_late'].value_counts(normalize=True)*100


In [ ]:
new['product_category_name'].value_counts()

In [ ]:
# consider only top categories and set others are 'others'

top_categories=new['product_category_name'].value_counts().nlargest(18).index

new['product_category_name']=new['product_category_name'].where(
    new['product_category_name'].isin(top_categories),
    other='others'
)

#check
new[new['product_category_name']=='others'].sort_values('delivery_time',ascending=False)
new.head(-10)